# Notebook to guide and create the BIDS directory for this dataset

1. Initialize folder structure with dcm2bids_scaffold before copying raw DICOM files
2. Copy raw DICOM files to sourcedata folder
3. Run dcm2bids (bash command) for each subject
4. Edit .jsons of the fmap files due to fmriprep legacy format for bids relative paths
5. Clear dot files

Requirements:
- dcm2bids version: 3.2.0
- dcm2niix version: v1.0.20241211

In [5]:
import os
import json

In [6]:
bids_dir = '/Volumes/T7/BIDS-MUSICNF'

# create bids_dir if not exists
if not os.path.exists(bids_dir):
    os.makedirs(bids_dir)

    # run dcm2bids_scaffold inside bids_dir
    os.system('dcm2bids_scaffold -o %s' % bids_dir)

INFO    | --- dcm2bids_scaffold start ---
INFO    | Running the following command: /opt/anaconda3/envs/musicnf-novelinterface/bin/dcm2bids_scaffold -o /Volumes/T7/BIDS-MUSICNF
INFO    | OS version: macOS-15.2-arm64-arm-64bit-Mach-O
INFO    | Python version: 3.13.0 | packaged by conda-forge | (main, Oct 17 2024, 12:32:35) [Clang 17.0.6 ]
INFO    | dcm2bids version: 3.2.0
INFO    | Checking for software update
INFO    | Currently using the latest version of dcm2bids.
INFO    | The files used to create your BIDS directory were taken from https://github.com/bids-standard/bids-starter-kit. 

INFO    | Tree representation of /Volumes/T7/BIDS-MUSICNF/
INFO    | /Volumes/T7/BIDS-MUSICNF/
INFO    | ├── code/
INFO    | ├── derivatives/
INFO    | ├── sourcedata/
INFO    | ├── tmp_dcm2bids/
INFO    | │   ├── log/
INFO    | │   │   ├── ._scaffold_20250125-165404.log
INFO    | │   │   └── scaffold_20250125-165404.log
INFO    | │   └── ._log
INFO    | ├── ._.bidsignore
INFO    | ├── ._CHANGES
INFO   

## Copy files manually following this structure:

```
bidspath
├── sourcedata
│   ├── 01
│   │   ├── 01 (optional session)
│   │   │   ├── dicom files
```

## dcm2bids

In [7]:
# get current path
curr_dir = os.getcwd()

In [ ]:
# Run dcm2bids (bash) for a single subject
subject_id = '03'

# run bash command for dcm2bids
os.system(f"dcm2bids \
          -d {bids_dir}/sourcedata/{subject_id} \
          -p {subject_id} \
          -c {curr_dir}/dcm2bids_config.json \
          -o {bids_dir}")

INFO    | --- dcm2bids start ---
INFO    | Running the following command: /opt/anaconda3/envs/musicnf-novelinterface/bin/dcm2bids -d /Volumes/T7/BIDS-MUSICNF/sourcedata/03 -p 03 -c /Users/alexandresayal/GitHub/musicnf-novelinterface/scripts/bids/dcm2bids_config.json -o /Volumes/T7/BIDS-MUSICNF
INFO    | OS version: macOS-15.2-arm64-arm-64bit-Mach-O
INFO    | Python version: 3.13.0 | packaged by conda-forge | (main, Oct 17 2024, 12:32:35) [Clang 17.0.6 ]
INFO    | dcm2bids version: 3.2.0
INFO    | dcm2niix version: v1.0.20241211
INFO    | Checking for software update
INFO    | Currently using the latest version of dcm2bids.
INFO    | Currently using the latest version of dcm2niix.
INFO    | participant: sub-03
INFO    | config: /Users/alexandresayal/GitHub/musicnf-novelinterface/scripts/bids/dcm2bids_config.json
INFO    | BIDS directory: /Volumes/T7/BIDS-MUSICNF
INFO    | Auto extract entities: False
INFO    | Reorder entities: True
INFO    | Validate BIDS: False

INFO    | Running: dcm

## Edit .jsons
So fmriPrep does not yet support the BIDS URL format - bids::sub-01/func/... in the IntendedFor field. So, what we need to do is remove the 'bids::sub-XX/' part from the IntendedFor field in the .json files of the fieldmap files.

In [18]:
# we should have three .json files in the fmap folder of this subject
fmap_dir = os.path.join(bids_dir, f'sub-{subject_id}', 'fmap')

# get all .json files in the fmap folder
fmap_files = [f for f in os.listdir(fmap_dir) if f.endswith('.json') and not f.startswith('.')]

print(fmap_files)

# check if they are three
if len(fmap_files) == 3:
    print('All three .json files are present in the fmap folder')

# for each of the files, load the file, read the IntendedFor field, edit the field by removing the first part, and save it back
for f in fmap_files:
    with open(os.path.join(fmap_dir, f), 'r') as file:
        data = json.load(file)
        data['IntendedFor'] = [x.split(f'bids::sub-{subject_id}/')[1] for x in data['IntendedFor']]
    with open(os.path.join(fmap_dir, f), 'w') as file:
        json.dump(data, file, indent=4)
        print(f'{f} has been edited')


['sub-02_magnitude1.json', 'sub-02_magnitude2.json', 'sub-02_phasediff.json']
All three .json files are present in the fmap folder
sub-02_magnitude1.json has been edited
sub-02_magnitude2.json has been edited
sub-02_phasediff.json has been edited


In [14]:
# Clear all dot files

os.system(f"dot_clean -m {bids_dir}")

0